# TP03 : Régression logistique

## Régression linéaire polynomiale 
Dans cette partie, comme expliqué en cours nous voyons un exemple simple d'un modèle linéaire sous forme d'un polynome multiple. 



In [ ]:
from sklearn.linear_model import LinearRegression  
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
X = [i for i in range(10)]
Y = [random.gauss(x,0.75) for x in X]

X = np.asarray(X)
Y = np.asarray(Y)

X = X[:,np.newaxis]
Y = Y[:,np.newaxis]

plt.scatter(X,Y)

In [ ]:
degree = 4 #degrès du polynome résultant.

polynomial_features = PolynomialFeatures(degree = degree)
X_TRANSF = polynomial_features.fit_transform(X)
model = LinearRegression()
model.fit(X_TRANSF, Y)

In [ ]:
Y_NEW = model.predict(X_TRANSF)

rmse = np.sqrt(mean_squared_error(Y,Y_NEW))
r2 = r2_score(Y,Y_NEW)

print('RMSE: ', rmse)
print('R2: ', r2)

In [ ]:
x_new_min = 0.0
x_new_max = 10.0

X_NEW = np.linspace(x_new_min, x_new_max, 100)
X_NEW = X_NEW[:,np.newaxis]

X_NEW_TRANSF = polynomial_features.fit_transform(X_NEW)

Y_NEW = model.predict(X_NEW_TRANSF)

plt.plot(X_NEW, Y_NEW, color='coral', linewidth=3)

plt.grid()
plt.xlim(x_new_min,x_new_max)
plt.ylim(0,10)

title = 'Degree = {}; RMSE = {}; R2 = {}'.format(degree, round(rmse,2), round(r2,2))

plt.title("Polynomial Linear Regression using scikit-learn and python 3 \n " + title,
          fontsize=10)
plt.xlabel('x')
plt.ylabel('y')
plt.scatter(X,Y)
plt.show()

## Régression logistique 

La régression logistique est généralement utilisée à des fins de classification. Contrairement à la régression linéaire, la variable à prédire ne peut prendre qu'un nombre limité de valeurs (valeurs discrètes). 

Lorsque le nombre de résultats possibles est seulement deux, on parle de régression logistique binaire.

![](img/logistic.JPG) 

Dans la figure ci-dessus on comprend que la régression logistique est composée d'une régression linéaire suivie de l'application d'une certaine fonction. Cette fonction est la fonction sigmoid dont voici le graphe : 

![](img/sigmoid.JPG) 


## 1 - Préparation des données : 
Les données consistent en un ensemble de notes des etudiants et la valeur à prédire est si l'etudiant est admis(1) ou pas(0) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
header = ["Note1", "Note2", "Admis"]
notes = pd.read_csv('datasets/marks.txt', names=header)
notes.head()

In [ ]:
X = notes.iloc[:, :-1]
y = notes.iloc[:, -1]

In [ ]:
admis = notes.loc[y == 1]
non_admis = notes.loc[y == 0]

plt.scatter(admis.iloc[:, 0], admis.iloc[:, 1], s=10, label='Admis')
plt.scatter(non_admis.iloc[:, 0], non_admis.iloc[:, 1], s=10, label='Non Admis')
plt.legend()
plt.show()

## 2- Régression logistique 

**Expressions mathématiques :  **:
<img src="img/Math.PNG" alt="Drawing" style="width: 300px;"/>
<img src="img/cost.PNG" alt="Drawing" style="width: 300px;"/>
<img src="img/total.PNG" alt="Drawing" style="width: 300px;"/>
<img src="img/deriv.PNG" alt="Drawing" style="width: 300px;"/>

In [ ]:
# TODO : Calculer le sigmoid de la valeur x 
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Test : 
sigmoid(0)

In [ ]:
# TODO : La fonction de coût utilisée dans la régression logistique 
def J(x,y,theta):
    z = np.dot(x,theta)
    h = sigmoid(z)
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

In [ ]:
theta = np.zeros((X.shape[1]+1,1))
#RESHAPE Y
y = y[:,np.newaxis]

print(X.shape)
print(y.shape)
print(theta.shape)

In [ ]:
from scipy.optimize import fmin_tnc
# TODO : Programmer la fonction d'entrainement du modèle 
def train(x,y, theta):
    lr = 0.01
    for i in range(10000):
        z = np.dot(x,theta)
        h = sigmoid(z)
        gradient = np.dot(x.T,(h-y)) / y.size
        theta -= lr * gradient
        
    return theta

# TODO : fonction de normalisation des données X 
def normalisation(X):
    mins = np.min(X, axis = 0)
    maxs = np.max(X, axis = 0)
    rng = maxs - mins
    norm_X = 1 - ((maxs - X)/rng)
    return norm_X 

# TODO : Entrainer le modèle en choisissant les bons hyperparamètres. 
norm_X = normalisation(X)
#AJOUTER des 1 pour le parametre theta-zero
norm_X= np.append(np.ones((norm_X.shape[0],1)),norm_X,axis=1)
theta = train(norm_X,y,theta)

theta

In [ ]:
x_values = np.array([np.min(X.values[:,1]),np.max(X.values[:,1])])
y_values = (- (theta[0] + np.dot(theta[1], x_values[0])) / theta[2],  - (theta[0] + np.dot(theta[1], x_values[1])) / theta[2])

plt.plot(x_values, y_values, label='ligne de décision')
plt.xlabel('Note 1 ')
plt.ylabel('Note 2 ')
plt.legend()
plt.show()

In [ ]:
# TODO : donner la fonction de prédiction qui retourne la probabilité que X est dans chaque classe 
def predict(x):
    z = np.dot(x,theta)
    h = sigmoid(z)
    if h>0.5:
        return 1
    else :
        return 0

## 3- Implementation sous sklearn : 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# TODO : Diviser les données en données d'entrainement et données de tests (Fait dans le TP02 )
## Décider de la taille des données pour chaque set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Créer le modèle 
model = LogisticRegression()

# Entraîner le modèle 
model.fit(X_train, y_train)

# Prédire les classes 
predicted_classes = model.predict(X_test)

# Calculer le score du modèle 
accuracy = accuracy_score(y_test,predicted_classes)

print('le score du modèle : ',accuracy)